# InterMineR Workshop Use Case



We are going to re-create the workflow we did using the web interface using the R API.

The basic steps are:  

1. Load the InterMine library and choose an InterMine to query.
2. Query 1: Diabetes Genes: Fetch a list of genes that are associated with diabetes
3. Query 2: PAX6 + Pancreas: Fetch a list of genes that have medium or high expression in the pancreas and are in our PAX6 targets list
4. Intersection: Find which genes are present in _both_ Query 1 and Query2.
5. GWAS: Compare the intersection of the previous query with results from GWAS studies.

### Getting started - Load InterMineR and choose an InterMine

Load the InterMine library. If it's not already installed, visit
https://bioconductor.org/packages/release/bioc/html/InterMineR.html and follow the instructions to install.


In [10]:
library(InterMineR)

We want to query human data - so let's look and see what InterMines are available: 

In [11]:
listMines()

BMAP 
               "https://bmap.jgi.doe.gov/bmapmine/" 
                                           BeanMine 
            "https://mines.legumeinfo.org/beanmine" 
                                         BovineMine 
           "http://genomes.missouri.edu/bovinemine" 
                                            CHOmine 
               "https://chomine.boku.ac.at/chomine" 
                                       ChickpeaMine 
        "https://mines.legumeinfo.org/chickpeamine" 
                                         CowpeaMine 
          "https://mines.legumeinfo.org/cowpeamine" 
                                            FlyMine 
                   "http://www.flymine.org/flymine" 
                                          GrapeMine 
         "http://urgi.versailles.inra.fr/GrapeMine" 
                                          HumanMine 
               "http://www.humanmine.org/humanmine" 
                                    HymenopteraMine 
     "http://hymenopteragenome.org/hymenopteramine" 
                                         IndigoMine 
              "http://www.cbrc.kaust.edu.sa/indigo" 
                                         LegumeMine 
          "https://mines.legumeinfo.org/legumemine" 
                                          MaizeMine 
"http://maizemine.rnet.missouri.edu:8080/maizemine" 
                                          MedicMine 
              "http://medicmine.jcvi.org/medicmine" 
                                          MitoMiner 
   "http://mitominer.mrc-mbu.cam.ac.uk/release-4.0" 
                                            ModMine 
        "http://intermine.modencode.org/release-33" 
                                          MouseMine 
               "http://www.mousemine.org/mousemine" 
                                            OakMine 
     "https://urgi.versailles.inra.fr/OakMine_PM1N" 
                                         PeanutMine 
          "https://mines.legumeinfo.org/peanutmine" 
                                          PhytoMine 
         "https://phytozome.jgi.doe.gov/phytomine/" 
                                           PlanMine 
              "http://planmine.mpi-cbg.de/planmine" 
                                            RatMine 
                   "http://ratmine.mcw.edu/ratmine" 
                                            RepetDB 
           "http://urgi.versailles.inra.fr/repetdb" 
                                            SoyMine 
             "https://mines.legumeinfo.org/soymine" 
                                         TargetMine 
   "https://targetmine.mizuguchilab.org/targetmine" 
                                          ThaleMine 
               "https://apps.araport.org/thalemine" 
                                          WheatMine 
        "https://urgi.versailles.inra.fr/WheatMine" 
                                           WormMine 
    "http://intermine.wormbase.org/tools/wormmine/" 
                                            XenMine 
                   "http://www.xenmine.org/xenmine" 
                                          YeastMine 
      "https://yeastmine.yeastgenome.org/yeastmine" 
                                      ZebrafishMine 
                         "http://zebrafishmine.org"

Okay, let's select HumanMine from the list:

In [12]:
humanMine <- listMines()["HumanMine"] #select humanmine
humanMine #print out the value to see what's inside

HumanMine 
"http://www.humanmine.org/humanmine"

Okay, now let's tell InterMineR that we want to use HumanMine for our queries.

**Important:** you'll need an API token for this part so you can access your HumanMine account. You can get your token by logging into [HumanMine](http://www.humanmine.org/) and going to the account details tab within MyMine. Cut and paste your token into the code below.

In [9]:
im <- initInterMine(mine=humanMine, "YOUR TOKEN HERE")

### First Query: Diabetes Genes

Our first query will be to select all human genes that are associate with diabetes. This will require two constraints:

1. Ensure all genes returned are `Home sapiens` genes (HumanMine contains some non-human genes for homology query purposes)
2. Restrict results to genes that are associated with `diabetes`.

In [17]:
query1Diabetes <- setQuery( 
  # here we're choosing which columns of data we'd like to see
  select = c("Gene.primaryIdentifier", "Gene.symbol"),
    # set the logic for constraints. The first constraint is the first path+operator+value, 
    # e.g. Gene.organism.name = Homo sapiens, and the second constraint is the combination 
    # of the second path+operator+value, e.g. Gene.diseases.name CONTAINS diabetes
  where = setConstraints(
    paths = c("Gene.organism.name", "Gene.diseases.name"),
    operators = c("=", "CONTAINS"),
    values = list("Homo sapiens","diabetes")
  )
)

Okay, we've set the query up, so now let's actually run it:

In [22]:
query1DiabetesResults <- runQuery(im,query1Diabetes)

# and let's print out the first few results to make sure it looks like we'd expect:
head(query1DiabetesResults)

Gene.primaryIdentifier,Gene.symbol
<chr>,<chr>
100188782,NIDDM4
1056,CEL
10644,IGF2BP2
11132,CAPN10
1234,CCR5
1493,CTLA4


### Query 2: Pax6 targets that have high expression in the Pancreas

This time we're creating another query, but with slightly more complex constraints. We're looking for genes that are in the public HumanMine list `PL_Pax6_Targets`, that are also expressed in the pancreas at a `High` or `Medium` level. 

We'll need a few more **constraints** than we did in Query 1:  
1. all `Gene`s should be in the list `PL_Pax6_Targets`
2. `Gene.proteinAtlasExpression.tissue.name` should be equal to `Pancreas`
3. `Gene.proteinAtlasExpression.level` should be set to `High` OR `Medium`. This will require two constraints, one for each of medium and high.

We'd also like to see a few more **columns** this time: 
1. The `Gene`'s `primaryIdentifier` and `symbol`
2. The following expression data from Protein Atlas: 
    - `Gene.proteinAtlasExpression.cellType`
    - `Gene.proteinAtlasExpression.level`
    - `Gene.proteinAtlasExpression.tissue.name`

In [25]:
# Create a new query
query2UpInPancreas = newQuery(
  #here we're choosing which columns of data we'd like to see
  view = c("Gene.primaryIdentifier",
             "Gene.symbol",
             "Gene.proteinAtlasExpression.cellType",
             "Gene.proteinAtlasExpression.level",
             "Gene.proteinAtlasExpression.tissue.name"
  ),
  # set the logic for constraints (see the function below for this to make sense)
  constraintLogic = "A and (B or C) and D"
)

# If we ran the query above, it'd show us *all* genes and their expression. 
# Let's narrow it down a little by constraining it to genes that are of interest
query2UpInPancreasConstraint = setConstraints(
  paths = c("Gene", 
            "Gene.proteinAtlasExpression.level", 
            "Gene.proteinAtlasExpression.level", 
            "Gene.proteinAtlasExpression.tissue.name"),
  operators = c("IN", rep("=", 2), "="),
  # each constraint is automatically given a code, allowing us to manipulate the 
  # logic for the constraint. 
  # Below, the constraints are set to codes A, B, C, D in order, 
  #  e.g. Code A: "Gene" should be "IN" the list named "PL_DiabetesGenes"
  #       Code B: "Gene.proteinAtlasExpression.level" should be equal to "Medium"
  #       Code C: "Gene.proteinAtlasExpression.level" should be equal to "High"
  #       Code D: "Gene.proteinAtlasExpression.tissue.name" should be equal to Pancreas"
  # 
  # Now, you might be thinking "how can the expression level be equal to both Medium AND High?"
  # and the answer is - it can't, but take a quick look at the constraintLogic we set earlier - 
  # (B or C) makes it clear that we want one or the other (but not, for instance, Low) 
  values = list("PL_Pax6_Targets", "Medium", "High", "Pancreas")
)

# Add the constraint to our expressed pancreas query (previously we just _defined_ the constraint)
query2UpInPancreas$where <- query2UpInPancreasConstraint

Remember, that was just setting up the query - we haven't run it yet

In [29]:
# Now we have the query set up the way we want, let's actually *run* the query! 
query2UpInPancreasResults <-  runQuery(im = im, qry = query2UpInPancreas)

# Show me the first few results please! 
head(query2UpInPancreasResults) 

Gene.primaryIdentifier,Gene.symbol,Gene.proteinAtlasExpression.cellType,Gene.proteinAtlasExpression.level,Gene.proteinAtlasExpression.tissue.name
<chr>,<chr>,<chr>,<chr>,<chr>
10097,ACTR2,exocrine glandular cells,Medium,Pancreas
10097,ACTR2,islets of Langerhans,Medium,Pancreas
10196,PRMT3,exocrine glandular cells,Medium,Pancreas
10196,PRMT3,islets of Langerhans,Medium,Pancreas
1121,CHM,exocrine glandular cells,Medium,Pancreas
1121,CHM,islets of Langerhans,Medium,Pancreas


### Intersection: Which genes overlap in Query1 and Query2?

Let's check which genes are in BOTH lists that we've created. To do this, we'll strip down the columns we have to just primary identifiers, and then run a list intersect function.

In [34]:
primaryIdentifiers.diabetes <- query1DiabetesResults[["Gene.primaryIdentifier"]]
primaryIdentifiers.pancreas <- query2UpInPancreasResults[["Gene.primaryIdentifier"]]

primaryIdentifiers.diabetes
print ("---")
primaryIdentifiers.pancreas 

intersect(primaryIdentifiers.diabetes,primaryIdentifiers.pancreas)

[1] "100188782" "1056"      "10644"     "11132"     "1234"      "1493"     
 [7] "1636"      "169026"    "169792"    "2056"      "208"       "26060"    
[13] "26191"     "2642"      "2645"      "2820"      "3077"      "3159"     
[19] "3172"      "346171"    "3557"      "3559"      "3569"      "359"      
[25] "3630"      "3643"      "3651"      "3667"      "3710"      "3767"     
[31] "387082"    "389692"    "3990"      "4544"      "4760"      "5078"     
[37] "50943"     "50982"     "51124"     "5167"      "5325"      "5444"     
[43] "5468"      "5506"      "551"       "554"       "5611"      "56729"    
[49] "57061"     "5770"      "640"       "6514"      "6648"      "6833"     
[55] "6927"      "6928"      "6934"      "7422"      "7466"      "8462"     
[61] "8660"      "9451"      "9479"      "9882"

[1] "---"


[1] "10097"  "10097"  "10196"  "10196"  "1121"   "1121"   "11340"  "114818"
  [9] "115286" "116039" "124540" "124540" "140609" "140609" "143098" "143098"
 [17] "143187" "144100" "144100" "145282" "145282" "1478"   "1478"   "171425"
 [25] "171425" "1848"   "201255" "2045"   "2045"   "2048"   "219402" "219402"
 [33] "22827"  "22827"  "22920"  "22920"  "23085"  "23133"  "23133"  "23200" 
 [41] "23328"  "23328"  "23414"  "23414"  "23613"  "2494"   "2494"   "25925" 
 [49] "2619"   "26468"  "26468"  "26610"  "26610"  "27072"  "27316"  "27316" 
 [57] "2736"   "2736"   "28964"  "2971"   "29880"  "3172"   "3187"   "3187"  
 [65] "3205"   "3217"   "3233"   "3233"   "3397"   "3397"   "340371" "340371"
 [73] "3491"   "359948" "359948" "3615"   "3615"   "3728"   "3728"   "374868"
 [81] "3842"   "3842"   "389072" "389072" "3983"   "4082"   "4082"   "4211"  
 [89] "4212"   "4212"   "4301"   "4756"   "476"    "4774"   "490"    "490"   
 [97] "5033"   "5080"   "5087"   "5087"   "5090"   "5090"   "51319"  "51319" 
[105] "5144"   "5150"   "51741"  "51741"  "51776"  "5195"   "5195"   "5339"  
[113] "5422"   "54674"  "54681"  "54681"  "54828"  "54828"  "55084"  "55084" 
[121] "55120"  "55120"  "55137"  "55137"  "55152"  "55152"  "5566"   "5566"  
[129] "55703"  "5607"   "57117"  "57117"  "5727"   "5727"   "57565"  "57565" 
[137] "57616"  "5862"   "5862"   "6210"   "63877"  "6428"   "6428"   "65125" 
[145] "657"    "6809"   "6809"   "6840"   "6902"   "6902"   "6907"   "6907"  
[153] "6928"   "6934"   "6934"   "6938"   "7110"   "7110"   "7444"   "7533"  
[161] "79767"  "79971"  "80700"  "8339"   "8339"   "83938"  "83989"  "83989" 
[169] "84193"  "84529"  "84618"  "84669"  "84669"  "84858"  "84858"  "8543"  
[177] "8543"   "8667"   "8667"   "8891"   "8891"   "9039"   "9039"   "905"   
[185] "905"    "93986"  "93986"  "9678"   "9686"   "9690"   "9690"   "9792"  
[193] "987"    "9922"

[1] "3172" "6928" "6934"